In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import os, sys
import toml
config = toml.load('../../../../configuration/input_configuration.toml')
%matplotlib inline

from IPython.display import display, HTML

if (sys.version_info < (3, 0)):
    HTML('''<script>
    code_show=true; 
    function code_toggle() {
     if (code_show){
     $('div.input').hide();
     } else {
     $('div.input').show();
     }
     code_show = !code_show
    } 
    $( document ).ready(code_toggle);
    </script>
    <form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')


In [2]:
working_dir = r'../../../../outputs/agg/dash'
# pd.options.display.float_format = '{:0,.0f}'.format
format_percent = "{:,.2%}".format

In [3]:
def load_data(fname):
    """ open and join model and survey data """

    df_model = pd.read_csv(os.path.join(working_dir,fname+'.csv'))
    df_model['source'] = 'model'
    df_survey = pd.read_csv(os.path.join(working_dir,'survey',fname+'.csv'))
    df_survey['source'] = 'survey'
    df = df_model.append(df_survey)

    return df

In [4]:
parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'],
        columns=['CountyName','ParcelID'])

In [5]:
def dist_by_mode(purpose=None):
    df = load_data('tour_distance')
    # Trim any unreasonable survey distances off by taking max distance as max modeled bin value
    max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
    df = df.loc[df['tautodist_bin'] <= max_model_dist]
    df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
    df_dist = pd.DataFrame()
    for source in ['model','survey']:
        if purpose is not None:
            _df = df[(df['source'] == source) & (df['pdpurp'] == purpose)]
        else:
            _df = df[(df['source'] == source)]
        df1 = _df.groupby('tmodetp').sum()[['wt_autodist']].reset_index()
        df2 = _df.groupby('tmodetp').sum()[['toexpfac']].reset_index ()
        _df = df1.merge(df2, on='tmodetp')
        _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
        _df['source'] = source
        df_dist = df_dist.append(_df)
    df = df_dist.pivot_table(index='tmodetp', columns='source', aggfunc='first', values='commute_distance')
    df.rename(columns={'tmodetp': 'Tour Mode'}, inplace=True)
    for col in ['School Bus',' Park']:
        if (col in df.index) & (purpose != 'School'):
            df.drop(col, axis=0, inplace=True)
        if (col != 'School Bus') & (col in df.index):
            df.drop(col, axis=0, inplace=True)
    
    return df

### Total

In [6]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df = df[(df['pdpurp'] == 'School')]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df = df.groupby('source').sum()
pd.DataFrame(df['wt_autodist']/df['toexpfac'], columns=['School Distance (miles)'])

,School Distance (miles)
source,
model,5.280159
survey,4.374166


### By Purpose

In [7]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source)]
    df1 = _df.groupby('pdpurp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pdpurp').sum()[['toexpfac']].reset_index ()
    _df = df1.merge(df2, on='pdpurp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pdpurp', columns='source', aggfunc='first', values='commute_distance')
df.rename(columns={'pdpurp': 'Tour Purpose'}, inplace=True)
# df.drop('Change Mode Inserted Purpose', axis=0, inplace=True)
df

source,model,survey
pdpurp,,
Escort,4.374003,5.329880
Meal,3.348192,3.160581
Personal Business,6.073374,6.161156
School,5.280159,4.374166
Shop,4.745518,4.127573
Social,6.032716,5.748173
Work,11.467482,11.335476


# Tour Distance
### All Tours

In [8]:
dist_by_mode()

source,model,survey
tmodetp,,
Bike,3.930136,1.930582
HOV2,6.967963,6.823707
HOV3+,6.927986,7.323668
Park,16.543251,15.126916
SOV,8.636261,8.290017
TNC,5.785582,6.131601
Transit,7.419351,7.888247
Walk,1.093580,0.852950


### Average Work Tour Distance By Mode

In [9]:
dist_by_mode('Work')

source,model,survey
tmodetp,,
Bike,5.027661,2.856404
HOV2,12.195181,11.434154
HOV3+,12.583342,14.403052
Park,16.543251,16.769421
SOV,11.871814,11.839207
TNC,7.139028,7.957826
Transit,13.504941,8.884595
Walk,0.759732,1.216414


### Average School Tour Distance by Mode

In [10]:
dist_by_mode('School')

source,model,survey
tmodetp,,
Bike,1.986532,0.816691
HOV2,4.822200,3.452323
HOV3+,5.607104,4.600548
SOV,6.117164,6.412966
School Bus,5.723207,3.815819
TNC,10.200000,9.895514
Transit,5.961485,7.486933
Walk,1.021002,0.554866


### Average Personal Business Tour Distance by Mode

In [11]:
dist_by_mode('Personal Business')

source,model,survey
tmodetp,,
Bike,3.383707,0.776547
HOV2,7.355747,5.483309
HOV3+,7.279415,5.386625
Park,NaN,11.721151
SOV,6.302153,6.325374
TNC,3.111055,3.785583
Transit,6.215218,10.811147
Walk,1.209750,1.449379


### Average Shopping Tour Distance by Mode

In [12]:
dist_by_mode('Shop')

source,model,survey
tmodetp,,
Bike,2.563701,0.994821
HOV2,5.955281,5.836525
HOV3+,6.073367,6.385363
SOV,4.822739,3.945021
TNC,2.460784,NaN
Transit,4.205089,2.754507
Walk,1.125457,0.696892


### Average Meal Tour Distance by Mode

In [13]:
dist_by_mode('Meal')

source,model,survey
tmodetp,,
Bike,2.527910,1.679894
HOV2,4.082779,3.310781
HOV3+,4.171551,3.665350
Park,NaN,16.113572
SOV,3.546391,3.364318
TNC,2.618611,1.511483
Transit,3.961114,3.821285
Walk,0.975113,1.666699


### Average Social Tour Distance by Mode

In [14]:
dist_by_mode('Social')

source,model,survey
tmodetp,,
Bike,4.146157,2.605522
HOV2,7.566524,7.504099
HOV3+,7.599793,8.009367
Park,NaN,13.553218
SOV,6.994786,5.993947
TNC,4.681720,3.067995
Transit,7.093901,8.818810
Walk,1.185024,0.737634


### Average Escort Tour Distance by Mode

In [15]:
dist_by_mode('Escort')

source,model,survey
tmodetp,,
Bike,5.850869,0.760243
HOV2,4.027690,6.229214
HOV3+,4.356728,5.533398
SOV,16.896744,4.479525
TNC,NaN,1.500000
Transit,2.227273,5.982501
Walk,1.075409,0.535637


Survey Sample Size Reference by Mode and Purpose

In [16]:
# Survey Sample size
df_tour_survey = pd.read_csv(r'../../../../inputs/base_year/survey/_tour.tsv', delim_whitespace=True)

In [17]:
df_tour_survey.pivot_table(index='tmodetp',columns='pdpurp', values='toexpfac', aggfunc='count')

pdpurp,1,2,3,4,5,6,7
tmodetp,,,,,,,
1,253.0,77.0,104.0,96.0,318.0,281.0,1135.0
2,136.0,22.0,16.0,9.0,18.0,25.0,107.0
3,1455.0,47.0,46.0,395.0,639.0,234.0,871.0
4,261.0,93.0,343.0,188.0,316.0,218.0,528.0
5,117.0,189.0,252.0,63.0,128.0,105.0,367.0
6,369.0,61.0,11.0,81.0,76.0,43.0,132.0
7,28.0,NaN,NaN,4.0,NaN,3.0,15.0
8,NaN,159.0,3.0,NaN,2.0,NaN,5.0
9,23.0,4.0,1.0,17.0,NaN,6.0,12.0


# Work Tour Distance by Worker Type

In [18]:
df = load_data('tour_distance')
# Trim any unreasonable survey distances off by taking max distance as max modeled bin value
max_model_dist = df.loc[(df['source']=='model'), 'tautodist_bin'].max()
df = df.loc[df['tautodist_bin'] <= max_model_dist]
df['wt_autodist'] = (df['tautodist_bin']+0.5)*(df['toexpfac'])
df_dist = pd.DataFrame()
for source in ['model','survey']:
    _df = df[(df['source'] == source) & 
             (df['pdpurp'] == 'Work') & 
             (df['pptyp'].isin(['Full-Time Worker','Part-Time Worker','University Student']))]
    df1 = _df.groupby('pptyp').sum()[['wt_autodist']].reset_index()
    df2 = _df.groupby('pptyp').sum()[['toexpfac']].reset_index()
    _df = df1.merge(df2, on='pptyp')
    _df['commute_distance'] = _df['wt_autodist']/_df['toexpfac']
    _df['source'] = source
    df_dist = df_dist.append(_df)
df = df_dist.pivot_table(index='pptyp', columns='source', aggfunc='first', values='commute_distance').reset_index()
df.rename(columns={'pptyp': 'Person Type'}, inplace=True)
df

source,Person Type,model,survey
0,Full-Time Worker,12.203979,12.478074
1,Part-Time Worker,8.130997,5.680146
2,University Student,6.024470,7.255794
